In [1]:
import pandas as pd
import numpy as np
import sklearn.decomposition
from scipy import stats
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
import emcee
import tissue_enrichment_analysis as tea
import matplotlib.patheffects as path_effects
import morgan as mg

import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

import networkx as nx
import matplotlib as mpl
from matplotlib import rc
rc('text', usetex=True)


ft = 35 #title fontsize

In [2]:
# Specify the genotypes
single_mutants = ['b', 'c', 'd', 'e', 'g']
double_mutants = {'a' : 'bd', 'f':'bc'}

thomas = mg.hunt('target_id', 'b', 'tpm', 'qval')
thomas.add_genmap('../input/library_genotype_mapping.txt', comment='#')
thomas.add_single_mutant(single_mutants)
thomas.add_double_mutants(['a', 'f'], ['bd', 'bc'])
thomas.set_qval()

# Add the tpm files: 
kallisto_loc = '../input/kallisto_all/'
thomas.add_tpm(kallisto_loc, '/kallisto/abundance.tsv', '')

# Make all possible combinations of WT, X
combs = {}
for gene in thomas.genmap.genotype.unique():
    if gene != 'wt':
        combs[gene] = 'WT_'+gene+'/'

# load all the beta values for each genotype:
sleuth_loc = '../sleuth/'
thomas.add_betas(sleuth_loc, 'betas.csv', combs)

thomas.filter_data(0, 0.1)

Number of na genes: 1005


In [3]:
# Perform Correlation Analysis
def tri_plot(matrix, xlabels, ylabels=[]):
    vmin= matrix.min().min()
    vmax= np.max(matrix).max()

    if len(ylabels) == 0:
        ylabels = xlabels
        
    mask = np.zeros_like(matrix)
    mask[np.tril_indices_from(mask)] = True
    with sns.axes_style("white"):
        ax = sns.heatmap(matrix, xticklabels=xlabels, yticklabels=ylabels, cmap='viridis',
                         mask=mask, square=True, vmin=vmin, vmax=vmax)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)


sydney = mg.brenner('interaction_analysis', thomas)


+ d d 977 13598 977 977 1.0
- d d 0 13598 977 977 1.32627934284e-33
+ g d 105 13598 1639 977 0.104330927453
- g d 108 13598 1639 977 0.172730285854
+ e d 658 13598 2457 977 1.0
- e d 32 13598 2457 977 1.40527737866e-48
+ b d 654 13598 1813 977 1.0
- b d 14 13598 1813 977 7.06324588473e-44
+ c d 166 13598 937 977 0.999999999997
- c d 57 13598 937 977 0.0968662790048
+ d g 105 13598 977 1639 0.104330927452
- d g 108 13598 977 1639 0.172730285853
+ g g 1639 13598 1639 1639 1.0
- g g 0 13598 1639 1639 1.38641321347e-98
+ e g 240 13598 2457 1639 4.97105094453e-05
- e g 224 13598 2457 1639 2.21037383387e-07
+ b g 182 13598 1813 1639 0.00222365320521
- b g 178 13598 1813 1639 0.000763749729053
+ c g 45 13598 937 1639 3.40273580741e-15
- c g 124 13598 937 1639 0.884460269638
+ d e 658 13598 977 2457 1.0
- d e 32 13598 977 2457 1.40527737868e-48
+ g e 240 13598 1639 2457 4.97105094455e-05
- g e 224 13598 1639 2457 2.21037383388e-07
+ e e 2457 13598 2457 2457 1.0
- e e 0 13598 2457 2457 5.480928